In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
import json

In [ ]:
f = open('/kaggle/input/coleridgeinitiative-show-us-the-data/train/0007f880-0a9b-492d-9a58-76eb0b0e0bd7.json')

In [ ]:
loaded = json.load(f)

In [ ]:
f.close()

In [ ]:
loaded[0]

In [ ]:
import nltk

In [ ]:
tagged_corpus = []
for i in range(20):
    sentences = loaded[i]['text']
    words  = nltk.word_tokenize(sentences)
    tagged_corpus.append(nltk.tag.pos_tag(words))

In [ ]:
len(tagged_corpus)

In [ ]:
sentences = []
for section in tagged_corpus:
    sentence = []
    for key in section:
        if key[0] != '.':
            sentence.append(key)
        else:
            sentences.append(sentence)
            sentence = []

In [ ]:
for x in loaded:
    print(x['section_title'])

In [ ]:
corp = nltk.corpus.brown.tagged_sents(categories='adventure')[:500]

In [ ]:
from nltk.util import unique_list

In [ ]:
tag_list = unique_list(tag for sent in sentences for (word, tag) in sent)

In [ ]:
syms = unique_list(word for sent in sentences for (word, tag) in sent)

In [ ]:
print(len(syms))

In [ ]:
print(len(tag_list))

In [ ]:
trainer = nltk.HiddenMarkovModelTrainer(tag_list, syms)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
len(sentences)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(sentences, range(150), test_size=0.1, shuffle=True, random_state=42)

In [ ]:
def train_and_test(est):
    hmm = trainer.train_supervised(x_train, estimator=est)
    print("%.2f"%(100 * hmm.evaluate(x_val)))

In [ ]:
trigrams = list(nltk.ngrams(words,n=3))

In [ ]:
tokenized =  nltk.tokenize.sent_tokenize(loaded[0]['text'])

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
docs = {}

In [ ]:
from glob import glob

In [ ]:
all_docs = {}

In [ ]:
i = 0

In [ ]:
sums = 0

In [ ]:
for i in all_docs:
    sums+=(all_docs[i][0]['section_title'] == 'Abstract')

In [ ]:
len(all_docs)

In [ ]:
sums

In [ ]:
for file in glob('/kaggle/input/coleridgeinitiative-show-us-the-data/train/*.json'):
#     print(file)
    f = open(file)
    all_docs["doc-"+str(i)] = json.load(f)
    f.close()
    i+=1

In [ ]:
docs = {}
i = 0

In [ ]:
for doc in  all_docs:
    sents = all_docs[doc][0]['text']
    tokenized = nltk.tokenize.sent_tokenize(sents)
    add = {"compound":0.0, "neg":0.0, "neu":0.0, "pos":0.0}
    for sent in tokenized:
        ss = sid.polarity_scores(sent)
        for so in sorted(ss):
            add[so]+=ss[so]
    for s in add:
        if len(tokenized):
            add[s]/=len(tokenized)
    docs[i] = add
    i+=1

In [ ]:
negs = []
posis = []
neus = []
compounds  = []
for key in docs:
    compounds.append(docs[key]['compound'])
    negs.append(docs[key]['neg'])
    posis.append(docs[key]['pos'])
    neus.append(docs[key]['neu'])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
def interpolate(compounds):
    # Define x, y, and xnew to resample at.
    x = np.array(range(len(compounds)))
    y = np.array(compounds)
    xnew = np.linspace(0, len(compounds) - 1, 100)

    # Define interpolators.
    f_linear = interp1d(x, y)
    f_cubic = interp1d(x, y, kind='cubic')

    # Plot.
    plt.plot(x, y, 'o', label='data')
    # plt.plot(xnew, f_linear(xnew), '-', label='linear')
    plt.plot(xnew, f_cubic(xnew), '-', label='cubic')
    plt.legend(loc='best')
    plt.show()

# Sentiments for Abstract/Introduction/etc
* Generally compounds means they cannot be properly identified and are tricky
* Negs--> negative, Posis-->Positives
* Neus --> Neutrals

In [ ]:
interpolate(compounds)

In [ ]:
interpolate(negs)

In [ ]:
interpolate(neus)

In [ ]:
interpolate(posis)

In [ ]:
def mad_based_outlier(points, thresh=3.5):
    points = np.array(points)
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh

In [ ]:
indexes = mad_based_outlier(compounds)

In [ ]:
compounds_outliers = []
negatives_outliers = []
positives_outliers = []
neutrals_outliers = []

In [ ]:
for i in range(len(indexes)):
    if indexes[i]:
        compounds_outliers.append(i)

In [ ]:
indexes_2 = mad_based_outlier(negs)
for j in range(len(indexes_2)):
    if indexes_2[j]:
        negatives_outliers.append(j)

In [ ]:
indexes = mad_based_outlier(posis)
for j in range(len(indexes)):
    if indexes[j]:
        positives_outliers.append(j)

In [ ]:
indexes_2 = mad_based_outlier(neus)
for j in range(len(indexes_2)):
    if indexes_2[j]:
        neutrals_outliers.append(j)

In [ ]:
def intersection(first, *others):
    return set(first).intersection(*others)